**Project Integration: SQL and Analytics Layer**

This notebook connects the machine learning output to an SQL database for structured querying, validation, and downstream analytics in Power BI.
It represents the deployment stage of the workflow, where model predictions transition from Python into a database environment to support business reporting and decision-making.

Workflow Overview
Model Results → SQL Database (SQLite) → Power BI Dashboard

In [1]:
# Load saved model and preprocessor pipeline
import joblib

model = joblib.load("best_svm.pkl")
preprocessor = joblib.load("preprocessor.pkl")

print("Model and preprocessor loaded successfully")

Model and preprocessor loaded successfully


In [2]:
# Load the X_test, X_test_prepared, and y_test dataset
X_test = joblib.load("X_test.pkl")
y_test = joblib.load("y_test.pkl")
X_test_prepared = joblib.load("X_test_prepared.pkl")

print("Dataset loaded successfully")

Dataset loaded successfully


**Generate Predictions on Test Data**

The test data (X_test, y_test, and already preprocessed X_test_prepared) will now be used to generate predicted classes and probabilities.

In [3]:
# Predict the data
y_pred = model.predict(X_test_prepared)
y_proba = model.predict_proba(X_test_prepared)[:,1]

In [4]:
# Combine results with original test data
predictions_df = X_test.copy()
predictions_df["actual_class"] = y_test.values
predictions_df["predicted_class"] = y_pred
predictions_df["abandonment_risk"] = y_proba

print("Predictions DataFrame created successfully")

Predictions DataFrame created successfully


In [5]:
# Display the prediction data
predictions_df.head()

,age,quantity,log_price,is_high_value,gender,city,category,device_type,os,day_of_week,month,actual_class,predicted_class,abandonment_risk
0,20,5,7.172532,1,Male,Mumbai,Apparel,Desktop,iOS,3,2,0,0,0.516295
1,56,5,7.280449,1,Female,New York,Home & Kitchen,Tablet,iOS,6,12,0,0,0.536322
2,48,1,6.466051,0,Male,London,Electronics,Tablet,iOS,2,8,1,1,0.500000
3,35,3,6.591825,0,Female,Sydney,Beauty & Personal Care,Mobile,Android,3,11,1,1,0.500000
4,59,4,5.658471,0,Female,Mumbai,Sports & Outdoors,Mobile,Windows,4,3,0,0,0.519962


This dataset now contains both the model’s predicted classes and risk probabilities, ready to be stored in SQL and analyzed in Power BI.

In [6]:
# Read source dataframes
import pandas as pd

customer_df = pd.read_csv("/content/customer_table.csv")
date_df = pd.read_csv("date_table.csv")
device_df = pd.read_csv("device_table.csv")
fact_df = pd.read_csv("fact_table.csv")
product_df = pd.read_csv("product_table.csv")

print("Data loaded successfully")

Data loaded successfully



Now we can connect to SQLite database. This will represent our analytics warehouse where Power BI can data pull from.

In [7]:
# Store all source tables
from sqlalchemy import create_engine

# Create database engine
engine = create_engine('sqlite:///cart_abandonment.db')

In [8]:
# Save project tables
customer_df.to_sql('customer_table', engine, index=False, if_exists='replace')
date_df.to_sql('date_table', engine, index=False, if_exists='replace')
device_df.to_sql('device_table', engine, index=False, if_exists='replace')
fact_df.to_sql('fact_table', engine, index=False, if_exists='replace')
product_df.to_sql('product_table', engine, index=False, if_exists='replace')

print("All raw tables successfully stored in SQL database")

All raw tables successfully stored in SQL database


In [9]:
# Check that all tables were stored properly
from sqlalchemy import text

with engine.connect() as conn:
    tables = conn.execute(text("SELECT name FROM sqlite_master WHERE type='table';"))
    print("Tables available in database:")
    for row in tables:
        print("-", row[0])

Tables available in database:
- customer_table
- date_table
- device_table
- fact_table
- product_table


Here we create a unified analytical view using SQL joins to build a complete dataset.

In [10]:
# Combine fact, customer, product, device, and date info
query = """
SELECT
    f.session_id,
    f.customer_id,
    f.product_id,
    f.device_id,
    f.date_id,
    f.quantity,
    f.abandonment_time,
    c.gender,
    c.age,
    c.city,
    d.device_type,
    d.os AS device_os,
    p.category AS product_category,
    p.price,
    dt.date
FROM fact_table f
LEFT JOIN customer_table c ON f.customer_id = c.customer_id
LEFT JOIN device_table d ON f.device_id = d.device_id
LEFT JOIN product_table p ON f.product_id = p.product_id
LEFT JOIN date_table dt ON f.date_id = dt.date_id;
"""

joined_df = pd.read_sql(query, engine)
print("Data joined successfully. Shape:", joined_df.shape)
joined_df.head()

Data joined successfully. Shape: (5000, 15)


,session_id,customer_id,product_id,device_id,date_id,quantity,abandonment_time,gender,age,city,device_type,device_os,product_category,price,date
0,1,979,20,5,252,4,7/13/2023,Female,20,London,Tablet,iOS,Beauty & Personal Care,1271.91,9/9/2023
1,2,373,12,3,353,2,11/7/2023,Female,24,New York,Mobile,Windows,Home & Kitchen,1450.64,12/19/2023
2,3,32,3,4,286,2,3/3/2023,Female,20,Mumbai,Mobile,Android,Electronics,866.21,10/13/2023
3,4,631,25,1,365,2,None,Male,50,Sydney,Tablet,iOS,Sports & Outdoors,243.63,12/31/2023
4,5,127,9,3,163,3,7/22/2023,Female,61,London,Mobile,Windows,Apparel,1302.14,6/12/2023


Now we insert our model’s predictions (predictions_df) into the same database for SQL analysis and Power BI.

In [11]:
# Save model predictions dataframe
predictions_df.to_sql("predictions_df", engine, index=False, if_exists="replace")
print("predictions_df saved successfully to SQL")

predictions_df saved successfully to SQL


We recreated a clean version of predictions table for Power BI. This fixes the issue in power BI failing to detect appropriate data type from SQLite database, by forcing all columns into readable data types.


In [12]:
with engine.connect() as conn:
    # Create a clean table with proper data types for Power BI
    conn.execute(text("""
        CREATE TABLE model_predictions AS
        SELECT
            CAST(age AS INTEGER) AS age,
            CAST(quantity AS INTEGER) AS quantity,
            CAST(log_price AS FLOAT) AS log_price,
            CAST(is_high_value AS INTEGER) AS is_high_value,
            CAST(gender AS TEXT) AS gender,
            CAST(city AS TEXT) AS city,
            CAST(category AS TEXT) AS category,
            CAST(device_type AS TEXT) AS device_type,
            CAST(os AS TEXT) AS os,
            CAST(day_of_week AS INTEGER) AS day_of_week,
            CAST(month AS INTEGER) AS month,
            CAST(actual_class AS INTEGER) AS actual_class,
            CAST(predicted_class AS INTEGER) AS predicted_class,
            CAST(abandonment_risk AS FLOAT) AS abandonment_risk
        FROM predictions_df;
    """))

    print("model_predictions table created successfully.")

model_predictions table created successfully.


We can now query our model outputs directly from SQL.

In [13]:
# Prediction summary by class
query_pred_summary = """
SELECT predicted_class, COUNT(*) AS count
FROM model_predictions
GROUP BY predicted_class;
"""
pd.read_sql(query_pred_summary, engine)

,predicted_class,count
0,0,211
1,1,539


The model predicts that most sessions (539 out of 750) are likely to result in cart abandonment (class 1), while fewer sessions (211) are predicted to complete purchases (class 0). This suggests cart abandonment is the dominant behavior in the dataset from our model prediction.

In [14]:
# Check average predicted abandonment risk
query_avg_risk = """
SELECT ROUND(
    AVG(abandonment_risk)*100, 2) AS avg_predicted_risk
FROM model_predictions;
"""
pd.read_sql(query_avg_risk, engine)

,avg_predicted_risk
0,50.2


In [15]:
# Export predictions for csv file
predictions_df.to_csv("predictions_results.csv", index=False)
print("Predictions exported to csv")

Predictions exported to csv
